In [45]:
import numpy as np
import joblib as jb
import json
from pathlib import Path
import uvicorn
from fastapi import FastAPI

app = FastAPI()

@app.get('/')
async def index():
    return {'text': 'Oiaaaa!!!'}

ROOT_PATH = Path().resolve().parent
scaler = jb.load(ROOT_PATH / 'preprocessed_data/scaler.pkl.z')
model = jb.load(ROOT_PATH / 'preprocessed_data/model.pkl.z')


def categorical_encoder(cat_name, cat_value):
    with open(ROOT_PATH / 'preprocessed_data/{}_encoder.json'.format(cat_name), 'r') as file:
        data = json.load(file)
    values = data.keys()
    return data[cat_value] if cat_value in data.keys() else int((values[0]+values[-1])/2)


# features order = ['Suburb', 'Rooms', 'Type', 'Postcode', 'stname', 'BuildingArea'] 
def inputTransformer(raw_list): 
    
    st = re.sub('\d+', '', raw_list[4]).strip(string.punctuation).strip()
    print(st)
    suburb = categorical_encoder('Suburb', raw_list[0])
    typ = categorical_encoder('Type', raw_list[2])
    postcode = categorical_encoder('Postcode', raw_list[3])
    stname = categorical_encoder('stname', st)
    return [suburb, raw_list[1], typ, postcode, stname, raw_list[5]]

def get_prediction(input_data):
    X = np.array(inputTransformer(input_data)).reshape(1, -1)
    X_poly = scaler.transform(X)
    y_hat = model.predict(X_poly)[0]
    return np.round(np.exp(y_hat), 2)

#if __name__ == '__main__':
#    uvicorn.run(app, port=5000, access_log=False)

In [10]:
import re
import string

In [48]:
inputTransformer(['Coburg', 2, 'u', '3058.0', 'Gladstone St ', 81.0])

Gladstone St


[140, 2, 1, 81, 2180, 81.0]

In [6]:
'4/34 Gladstone St'.replace('\d+', '')

'4/34 Gladstone St'

In [32]:
re.sub('\d+', '', '4/34 Gladstone St').strip(string.punctuation).strip()

'Gladstone St'

In [31]:
re.sub(r's', '', '4/34 Gladstone St')

'/ Gladstone St'

In [42]:
string.punctuation + '[d+]'

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~[d+]'

In [43]:
'4/34 Gladstone St'.strip(string.punctuation).strip()

'4/34 Gladstone St'